In [2]:
from typing import Any
from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf
from dotenv import load_dotenv
import io,glob,os
from langchain.schema.messages import HumanMessage
from langchain_openai import ChatOpenAI
from PIL import Image
import base64

[nltk_data] Error loading averaged_perceptron_tagger_eng: <urlopen
[nltk_data]     error [SSL: CERTIFICATE_VERIFY_FAILED] certificate
[nltk_data]     verify failed: unable to get local issuer certificate
[nltk_data]     (_ssl.c:997)>
[nltk_data] Error loading punkt_tab: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


In [1]:
%pip install unstructured pdfminer.six==20221105 pi_heif unstructured_inference pdf2image numpy==1.26.4 unstructured_pytesseract


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip uninstall numpy -y



Found existing installation: numpy 2.2.3
Uninstalling numpy-2.2.3:
  Successfully uninstalled numpy-2.2.3
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
os.environ["PATH"] += os.pathsep + "/opt/homebrew/bin"
os.environ["UNSTRUCTURED_OCR_AGENT"] = "pytesseract"

In [6]:
!which pdfinfo


/opt/homebrew/bin/pdfinfo


In [4]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
load_dotenv()
llm=ChatOpenAI(model="gpt-4-vision-preview",max_tokens=1024)
rpe=partition_pdf(filename="/Users/arun/Desktop/Darwin/Psycore/scripting/jupyter_testing/22-036458-01_GIS_early_process_evaluation_Accessible_CLIENT_USE.pdf",extract_images_in_pdf=True, infer_table_structure=True,chunking_strategy="by_title",max_characters=4000,new_after_n_chars=3800,combine_text_under_n_chars=2000)

ValueError: unrecognized chunking strategy 'title'

In [ ]:

def image2base64(image_path):
    with Image.open(image_path) as image:
        buffer=io.BytesIO()
        image.save(buffer,format=image.format)
        img_str=base64.b64encode(buffer.getvalue())
        return img_str.decode("utf-8")


In [ ]:
    
cwd="scripting/jupyter_testing/extracted_images"
files=glob.glob(cwd+"/*.jpg")
for file in files:
    image_str=image2base64(file)
    response=llm.invoke(
        [
            HumanMessage(
                content=[
                    {"type":"text","text":"please give a summary of the image provided , be descriptive and smart"},
                    {"type":"image_url","image_url":
                     {
                        "url":f"data:image/jpg;base64,{image_str}"

                    },
                    },
                ]
            )
        ]
    )
    print(file.split("/")[-1])
    print(response.content)
    print("========================================")